# Exploring Combined Data frames

This data frame is looking at the large data frames that were created by recording the FDR significant pearson correlation between EGFR and all proteins. The perpose of this notebook is to find genes that either have different significant correlations (pos/neg) or have the same correlation, but only in some cancers.  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import cptac
import cptac.utils as u
import plot_utils as p 

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        if pd.isnull(item):
            continue
        if item < -0.3:
            hasNeg = True
        if item > 0.3:
            hasPos = True
            
    if hasPos & hasNeg:
        return True
    return False

def Pvalue_sig(row):
    numSig = 0

    for item in row:
        if pd.isnull(item):
            continue
        if item < 0.05:
            numSig += 1
            
    return numSig

def CountPosNeg(row):
    hasPos = False
    hasNeg= False
    counter = 0
    for item in row:
        if pd.isnull(item):
            continue
        if item < -0.3:
            hasNeg = True
            counter += 1
        if item > 0.3:
            hasPos = True
            counter += 1
    return counter

Read in the wide version of the data frame (FDR all proteins) then set comparison to be the index. Then filter the data frame to only hae the correlation columns  

In [3]:
prot_FDR = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_all_FDR_wide.csv")
prot_FDR = prot_FDR.drop(['Unnamed: 0'], axis=1)
prot_FDR = prot_FDR.set_index('Comparison')
prot_FDR = prot_FDR.drop(['EGFR_proteomics'], axis=0)
prot_FDR

,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_colon,P_value_colon,Correlation_Lscc,P_value_Lscc
Comparison,,,,,,,,,,,,,,,,
PHLDA1_proteomics,0.816848,6.553435e-25,NaN,NaN,NaN,NaN,0.364797,0.000190,NaN,NaN,0.664271,3.426615e-15,NaN,NaN,0.709812,3.919195e-18
GRB2_proteomics,-0.610889,1.886384e-11,NaN,NaN,NaN,NaN,NaN,NaN,-0.302439,0.001321,-0.532341,2.559824e-09,NaN,NaN,NaN,NaN
SOCS2_proteomics,0.562720,1.343464e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478574,2.191662e-04
CDH4_proteomics,0.559180,1.790048e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLA2G15_proteomics,-0.556624,2.197562e-09,-0.298029,0.001566,NaN,NaN,0.274185,0.003025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SLIT3_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.291391,2.011415e-03
AAK1_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.290954,2.044509e-03
COMMD3-BMI1_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.290800,2.056267e-03


# Positive and Negative Correlations 

Filter data frame to only have correlation columns. Then use HasPosNeg function to add column with T/F values to record whether or not a gene has both positive or negavtive correlations. 

In [4]:
col = ["Correlation_Gbm","Correlation_kidney","Correlation_Ovar","Correlation_Brca","Correlation_Luad","Correlation_hnscc","Correlation_colon","Correlation_Lscc"]
FDR_corr = prot_FDR[col]
FDR_corr["Pos_Neg"] = FDR_corr.apply(HasPosNeg, axis = 1)



/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Filter data frame to have only proteins that have both pos and neg correlations. Then save file as csv file to be used as enrichment anaylsis notebook

In [5]:
FDR_corr_True = FDR_corr[FDR_corr['Pos_Neg']==True]
FDR_corr_True.to_csv("csv_files/pancan_EGFR_all_pos_neg_FDR.csv")
FDR_corr_True

,Correlation_Gbm,Correlation_kidney,Correlation_Ovar,Correlation_Brca,Correlation_Luad,Correlation_hnscc,Correlation_colon,Correlation_Lscc,Pos_Neg
Comparison,,,,,,,,,
DAB2_proteomics,-0.556402,NaN,NaN,0.326055,NaN,NaN,NaN,NaN,True
CTSC_proteomics,-0.546285,-0.302316,NaN,0.266940,0.30276,NaN,NaN,NaN,True
SCPEP1_proteomics,-0.531494,-0.386583,NaN,0.399187,NaN,NaN,NaN,NaN,True
FAM129B_proteomics,-0.514984,NaN,NaN,0.344093,NaN,NaN,NaN,0.353201,True
PPP1R18_proteomics,-0.497202,NaN,NaN,0.359142,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...
CRABP1_proteomics,NaN,NaN,NaN,NaN,NaN,-0.325890,0.438457,NaN,True
FBXO6_proteomics,NaN,NaN,NaN,NaN,NaN,-0.321105,0.354540,NaN,True
UVRAG_proteomics,NaN,NaN,NaN,NaN,NaN,-0.304493,0.376937,NaN,True


# Positive or Negative 

 Filter data frame to include only proteins where Pos_Neg column is false. Use CountPosNeg to count the number of columns that have correlations. 

In [6]:
FDR_corr_False = FDR_corr[FDR_corr['Pos_Neg']==False]
FDR_corr_False["Num_corr"] = FDR_corr_False.apply(CountPosNeg, axis = 1)
FDR_corr_False

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Correlation_Gbm,Correlation_kidney,Correlation_Ovar,Correlation_Brca,Correlation_Luad,Correlation_hnscc,Correlation_colon,Correlation_Lscc,Pos_Neg,Num_corr
Comparison,,,,,,,,,,
PHLDA1_proteomics,0.816848,NaN,NaN,0.364797,NaN,0.664271,NaN,0.709812,False,4
GRB2_proteomics,-0.610889,NaN,NaN,NaN,-0.302439,-0.532341,NaN,NaN,False,3
SOCS2_proteomics,0.562720,NaN,NaN,NaN,NaN,NaN,NaN,0.478574,False,2
CDH4_proteomics,0.559180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1
PLA2G15_proteomics,-0.556624,-0.298029,NaN,0.274185,NaN,NaN,NaN,NaN,False,1
...,...,...,...,...,...,...,...,...,...,...
SLIT3_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.291391,False,0
AAK1_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.290954,False,0
COMMD3-BMI1_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.290800,False,0


Sort Data frame with the Num_corr values descending. The filter out proteins that only had 0 or 1 correlation. Save file as csv to be used in enrichment anaylsis notebook 

In [7]:
FDR_corr_False.sort_values(by=['Num_corr'],ascending=False)
FDR_corr_False = FDR_corr_False[FDR_corr_False['Num_corr'].isin([2,3,4,5,6])]

In [8]:
FDR_corr_False.to_csv("csv_files/pancan_EGFR_all_FDR_atleast2_same_corr.csv")  